# Continação Pandas

Nessa aula vamos continuar vendo sobre a biblioteca Pandas e mais algumas das suas utilidades. 

## Sumário
- [Tipos de dados em tabelas](#Tipos-de-dados-em-tabelas)
    * [Strings](#Strings)
    * [Datetime e Timedelta](#Datetime-e-Timedelta)
- [Query](#Query)
- [Merge](#Merge)
- [Group By](#Group-By)
- [Conclusão](#Conclusão)

## Importando a biblioteca e lendo os dados 

Nessa aula usaremos três datasets:

- WorldCup: contém informações sobre as copas do mundo, como sede, ano, campeão e quantidade de gols
- WorldCupMatches: contém informações sobre todas as partidas que ocorreram em cada uma das copas, como data, estádio, placar e quantidade de gols
- WorldCupPlayers: contém informações sobre os jogadores que já participaram de uma copa do mundo, como nome, seleção, camisa e posição

Vamos ver um pouco mais sobre cada uma das tabelas abaixo.

In [ ]:
import pandas as pd

cups = pd.read_csv('WorldCups.csv')
matches = pd.read_csv('WorldCupMatches.csv')
players = pd.read_csv('WorldCupPlayers.csv')

### Overview da tabela de copas

In [ ]:
cups.head()

In [ ]:
cups.dtypes

### Overview da tabela de partidas

In [ ]:
matches.head()

In [ ]:
matches.dtypes

### Overview da tabela de jogadores

In [ ]:
players.head()

In [ ]:
players.dtypes

## Tipos de dados em tabelas
Além dos tipos de dados que já vimos, é possível lidar com alguns outros tipo bem úteis em tabelas, como strings e datetimes.

### Strings

Lidar com strings em Dataframes ou em Series é, na verdade, bem simples. Basta selecionar os dados e utilizar '.str' em seguida. Assim é possível tratar os dados como se fossem uma string só e, então, utilizar os metódos conhecidos para strings.

Por exemplo, vimos na tabela de jogadores que os seus nomes e dos técnios estão escritos com o nome em letras minúsculas e o sobrenome em letras maiúsculas. Digamos que gostaríamos de mudar isso para que tenhamos apenas a primeira letra de cada nome maiúscula.

In [ ]:
players['Coach Name'] = players['Coach Name'].str.title()
players['Player Name'] = players['Player Name'].str.title()

In [ ]:
players.head()

Agora temos um problema: o país do técnico deveria estar em letras maiúsculas. Vamos aproveitar para criar uma nova coluna para esse nome. 

Para isso vamos dividir o nome do técnico nos espaços e pegar o útlimo valor. Em seguida vamos retirar os parênteses e transformar as letras em maiúsculas.

In [ ]:
players['Coach County'] = players['Coach Name'].str.split(' ').str[-1]
players['Coach County'] = players['Coach County'].str.replace('(', '')
players['Coach County'] = players['Coach County'].str.replace(')', '')
players['Coach County'] = players['Coach County'].str.upper()

In [ ]:
players.head()

Essa são apenas algumas das utilidades de strings em tabelas, [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.capitalize.html) é possível ver mais algumas delas.

### [Datetime e Timedelta](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)

Esses tipos de dados, como os nomes sugerem, são utilizados para lidar com marcações de tempo.

Um datetime consiste de um dia, horário e fuso horário específicos. Enquanto isso, um timedelta é uma duração de tempo.

Na tabela de partidas, por exemplo, temos um atributo chamado Datetime, que consiste da data e horário em que a partida teve início. Primeiramente precisamos transformar os dados no tipo Datetime de fato e para isso é preciso especificar o formato da data.

In [ ]:
matches['Datetime'] = matches['Datetime'].str.replace('June', 'Jun').str.replace('July', 'Jul')
matches['Datetime'] = pd.to_datetime(matches['Datetime'], format = '%d %b %Y - %H:%M ')

Já existe uma coluna com o ano da partida, mas utilizando o datetime, podemos criar colunas com o respectivo dia e o mês de ocorrência.

Analogamente ao acesso de strings, precisamos utilizar '.dt' para indicar que estamos lidando com um datetime e, assim, acessar seus métodos e atributos.

In [ ]:
matches['Day'] = matches['Datetime'].dt.day
matches['Month'] = matches['Datetime'].dt.month

In [ ]:
matches[['Day', 'Month']].head()

## Query

Na [primeira aula de Pandas](https://github.com/icmc-data/Intro-DS-2020.1/blob/master/Aula2/introducao_pandas.ipynb) foi explicado como utilizar indexação booleana para acessar apenas partes de uma tabela. Vamos ver agora como utilizar a função query, que também tem como finalidade realizar acessos. Essa função é mais eficiente e em geral mais legível do que a indexação booleana.

Para utilizá-la, é passado como parâmetro a expressão utilizada para filtrar a tabela. Nessa expressão são utilizados os nomes dos atributos, os operadores de comparação (>, >=, <, <=, ==) e os valores desejados e, caso o nome do atributo tenha espaços, ele deve ser escrito entre crases.

Um exemplo seria selecionar todos os jogos da seleção brasileira:

In [ ]:
# Com indexação binária
matches[(matches['Home Team Name'] == 'Brazil') | (matches['Away Team Name'] == 'Brazil')]

In [ ]:
# Com Query
matches.query('(`Home Team Name` == "Brazil") or (`Away Team Name` == "Brazil")')

Vamos ver quantos desses jogos o Brasil ganhou, ou seja, em quantos dos jogos era mandante e o mandante fez mais gols ou era convidado e o convidado fez mais gols:

In [ ]:
matches.query('(`Home Team Name` == "Brazil" and `Home Team Goals` > `Away Team Goals`) or  \
              (`Away Team Name` == "Brazil" and `Away Team Goals` > `Home Team Goals`)')

## Merge

Essa função tem como objetivo juntar tabelas. Existem algumas maneiras de fazer isso, uma vez que nem sempre todos os dados serão compatíveis.

![Tipos de merge](./src/merge.jpg)

- Left: todos os valores da tabela da esquerda são trazidos, mas da tabela da direita são selecionados apenas aqueles que tem um *match* na tabela da esquerda
- Right: todos os valores da tabela da direita são trazidos, mas da tabela da esqurda são selecionados apenas aqueles que tem um *match* na tabela da direita
- Inner: são selecionadas apenas os valores que estão tanto na tabela da esquerda quanto na da direita
- Outer: todos os valores são selecionados, mesmo se não houver um *match*

Estamos lidando nesse notebook com três datasets separados, mas podemos juntá-los de alguma maneira. Não precisamos nos preocupar muito com a forma (left, right, inner ou outer) que será feito o merge, pois temos garantia de que os dados são consistentes, uma vez que todo jogador da tabela players participou de uma partida válida da tabela matches e todas essas partidas pertencem a uma copa identificada na tabela cups. Isso significa que para todos as linhas de uma tabela haverá um match na outra.

Vamos começar juntando a tabela das copas com a tabela de partidas. Para isso, podemos utilizar o atributo 'ano' que ambas tem em comum.

In [ ]:
matches_cups = pd.merge(matches, cups, on='Year')

In [ ]:
matches_cups.head()

Podemos selecionar agora o time que ficou com a quinta posição em cada uma das copas. Para isso temos que descobrir que time perdeu para o campeão nas quartas de final.

In [ ]:
fifth_away = matches_cups.query('Stage == "Quarter-finals" and (`Home Team Name` == Winner)')

In [ ]:
fifth_away = fifth_away[['Year', 'Away Team Name']]
fifth_away

Note que na query acima temos linhas duplicadas que não nos interessam, então podemos retirar uma delas utilizando a função **drop_duplicates**

In [ ]:
fifth_away = fifth_away.drop_duplicates(['Year', 'Away Team Name'])

In [ ]:
fifth_home = matches_cups.query('Stage == "Quarter-finals" and `Away Team Name` == Winner')

In [ ]:
fifth_home = fifth_home[['Year', 'Home Team Name']]
fifth_home

Podemos juntar as duas tabelas utilizando a função **concat**:

In [ ]:
pd.concat([fifth_away, fifth_home])

Outro exemplo de junção de tabelas é utilizando as partidas e os jogadores. Essas tabelas tem em comum o atributo 'MatchID':

In [ ]:
matches_players = pd.merge(players, matches, on='MatchID')

In [ ]:
matches_players.head()

## Group By

Esse tipo de operação é utilizado para agrupar dados. Com ela podemos responder, por exemplo, as seguintes perguntas:

- Qual a média de gols por copa?
- Para cada time, qual a média de gols por copa como visitante? E como mandante?
- Para cada jogador, quantos gols foram feitos em jogos em que ele participou?
- Quantos jogos aconteceram em cada mês?

Na prática sua utilização é a seguinte: seleciona-se o atributo da tabela que será utilizado para agrupar os dados e em seguida a operação (soma, média, desvio padrão...) e as colunas nas quais será feita na agregação.

Qual a média de gols por copa?


In [ ]:
matches.groupby('Year')[['Home Team Goals', 'Away Team Goals']].mean()

Para cada time, qual a média de gols por copa como mandante?


In [ ]:
matches.groupby(['Year', 'Home Team Name'])[['Home Team Goals']].mean()

Para cada time, qual a média de gols por copa como visitante?

In [ ]:
matches.groupby(['Year', 'Away Team Name'])[['Away Team Goals']].mean()

Para cada jogador, quantos gols foram feitos em jogos em que ele participou no time visitante?

In [ ]:
matches_players.groupby(['Player Name'])[['Player Name', 'Away Team Goals']].sum()

Para cada jogador, quantos gols foram feitos em jogos em que ele participou no time mandante?

In [ ]:
matches_players.groupby(['Player Name'])[['Player Name', 'Home Team Goals']].sum()

Para cada jogador, quantos gols foram feitos em jogos em que ele participou em geral

In [ ]:
matches_players.groupby(['Player Name'])[['Home Team Goals', 'Away Team Goals']].sum().sum(axis = 1)

Quantos jogos aconteceram em cada mês?

Nesse caso podemos utilizar a função **value counts**. Ela conta, para cada um dos valores existentes, a sua frequência na tabela, que é justamente o que queremos.

In [ ]:
matches['Month'].value_counts()

## Conclusão

Por hoje é isso, galera. Sempre bom aprender um pouco mais de Pandas e das coisas que podemos fazer utilizando-a. Recomendo dar uma olhada não só nos assuntos abordados aqui, mas em outras funções dessa biblioteca.

Além disso, recomendo fazer os exercícios disponibilizados. Qualquer dúvida é só entrar em contato =D